<a href="https://colab.research.google.com/github/kjh50400/Dacon/blob/master/KEA_power_consumption_pred_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ```
 function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}

setInterval(ClickConnect, 180 * 1000) // 60초 마다 함수 실행
 ```

## 저장 및 제출용 파일 만들기

예시

```python
TODAY = datetime.now(timezone('Asia/Seoul')).strftime("%y%m%d") + '_'

submission.to_csv('./submission/'+TODAY+'XGB_1.csv', encoding='utf-8-sig')
```

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 모듈
import os, pathlib, glob
import random
from pytz import timezone # Colab 사용 시, Asia/Seoul로 지정 필요
from datetime import datetime # 학습 소요 시간 측정

import numpy as np
import pandas as pd

import sklearn as sk

# import tensorflow as tf
# import keras # tf.keras와 그냥 keras를 혼용 시 문제가 발생할 수 있다(?). 하나로 통일해 주자.
# import torch
# import torchvision
# import cv2

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

In [3]:
# # matplotlib 한글 사용을 위한 폰트 설치
# !apt-get install fonts-nanum*
# !apt-get install fontconfig
# !fc-cache -fv
# !cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
# !rm -rf /content/.cache/matplotlib/*

In [4]:
# import matplotlib as mpl
# import matplotlib.font_manager as fm  

# mpl.rcParams['axes.unicode_minus'] = False
# # 그래프에서 마이너스 폰트 깨질 경우 대비

# path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
# font_name = fm.FontProperties(fname=path, size=18).get_name()
# plt.rc('font', family=font_name)
# fm._rebuild() # 이걸 해줘야 plt.rc가 작동

In [5]:
# 버전 확인 및 GPU 사용 가능 여부 확인
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)
print()
print('Scikit-learn: ', sk.__version__)
print()
# print('Tensorflow: ', tf.__version__)
# print('Keras: ', keras.__version__)
# print('torch: ', torch.__version__)
# print('torchvision: ', torchvision.__version__)
# print('OpenCV: ', cv2.__version__)
# print()
# print(tf.config.list_physical_devices('GPU'))

Numpy:  1.19.5
Pandas:  1.1.5

Scikit-learn:  0.22.2.post1



In [6]:
# 같은 결과 값을 얻기 위한 seed 고정
SEED = 10

random.seed(SEED)
np.random.seed(SEED)
# tf.random.set_seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed_all(SEED) # GPU 상의 torch seed 고정?

In [7]:
# 작업경로 변경
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Power_Consumption_Pred')

print('Current directory: ', os.getcwd())

Current directory:  /content/gdrive/My Drive/Colab Notebooks/Power_Consumption_Pred


# EDA

In [21]:
df_train = pd.read_csv('./energy/train.csv', encoding='CP949')

print(df_train.info())
print('='*100)
display(df_train.describe())
print('='*100)
display(df_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   num         122400 non-null  int64  
 1   date_time   122400 non-null  object 
 2   전력사용량(kWh)  122400 non-null  float64
 3   기온(°C)      122400 non-null  float64
 4   풍속(m/s)     122400 non-null  float64
 5   습도(%)       122400 non-null  float64
 6   강수량(mm)     122400 non-null  float64
 7   일조(hr)      122400 non-null  float64
 8   비전기냉방설비운영   122400 non-null  float64
 9   태양광보유       122400 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 9.3+ MB
None


,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000


,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [22]:
df_test = pd.read_csv('./energy/test.csv', encoding='CP949')

print(df_test.info())
print('='*100)
display(df_test.describe())
print('='*100)
display(df_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   num           10080 non-null  int64  
 1   date_time     10080 non-null  object 
 2   기온(°C)        3360 non-null   float64
 3   풍속(m/s)       3360 non-null   float64
 4   습도(%)         3360 non-null   float64
 5   강수량(mm, 6시간)  1680 non-null   float64
 6   일조(hr, 3시간)   3360 non-null   float64
 7   비전기냉방설비운영     2296 non-null   float64
 8   태양광보유         1624 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 708.9+ KB
None


,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
count,10080.000000,3360.000000,3360.000000,3360.000000,1680.000000,3360.000000,2296.0,1624.0
mean,30.500000,27.805804,2.440327,81.963095,2.191845,0.612917,1.0,1.0
std,17.318961,2.378122,1.835905,11.900843,6.786772,0.929587,0.0,0.0
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,1.0,1.0
25%,15.750000,26.000000,1.100000,74.000000,0.000000,0.000000,1.0,1.0
50%,30.500000,27.500000,2.100000,83.000000,0.000000,0.000000,1.0,1.0
75%,45.250000,29.200000,3.200000,92.000000,0.900000,1.000000,1.0,1.0
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.0,1.0


,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_sub = pd.read_csv('./energy/sample_submission.csv', encoding='CP949')

print(df_sub.info())
print('='*100)
display(df_sub.describe())
print('='*100)
display(df_sub)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num_date_time  10080 non-null  object
 1   answer         10080 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 157.6+ KB
None


,answer
count,10080.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


,num_date_time,answer
0,1 2020-08-25 00,0
1,1 2020-08-25 01,0
2,1 2020-08-25 02,0
3,1 2020-08-25 03,0
4,1 2020-08-25 04,0
...,...,...
10075,60 2020-08-31 19,0
10076,60 2020-08-31 20,0
10077,60 2020-08-31 21,0
10078,60 2020-08-31 22,0


In [24]:
10080/60/24

7.0